In [1]:
!pip install diffusers transformers accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 39.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler, StableDiffusionXLPipeline
from diffusers.loaders import LoraLoaderMixin
import torch
import os

import matplotlib.pyplot as plt
import random
from PIL import Image
import numpy as np

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# model_id = "runwayml/stable-diffusion-v1-5"
model_id = "stabilityai/stable-diffusion-xl-base-1.0"  # SDXL for higher quality
# model_id = "SG161222/Realistic_Vision_V5.1_noVAE"  # Good for realistic architecture
# model_id = "prompthero/openjourney-v4"  # Good for stylized buildings

# Set up the pipe with better defaults
pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    safety_checker=None,  # Disable safety checker for architectural images
    requires_safety_checker=False
)

# Improve the scheduler for better quality
pipe.scheduler = DPMSolverMultistepScheduler.from_config(
    pipe.scheduler.config,
    algorithm_type="dpmsolver++",
    use_karras_sigmas=True
)

pipe = pipe.to("cuda")
pipe.enable_attention_slicing()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.78G [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/10.3G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

In [5]:
import os
save_dir = '/content/drive/MyDrive/11785 Intro to DL/Final Projects/set_4'
os.makedirs(save_dir, exist_ok=True)


## Prompt

In [6]:
# Load SDXL model
print("Loading model...")
pipe = StableDiffusionXLPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    use_safetensors=True
)

# Improve the scheduler
pipe.scheduler = DPMSolverMultistepScheduler.from_config(
    pipe.scheduler.config,
    algorithm_type="dpmsolver++",
    use_karras_sigmas=True
)

# Move to GPU and optimize
pipe = pipe.to("cuda")
pipe.enable_attention_slicing()
print("Model loaded and ready!")

Loading model...


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Model loaded and ready!


In [7]:
import random

# Comprehensive building attributes for variety
building_types = [
    # None:
    "","",

    # Commercial
    "office tower", "corporate headquarters", "commercial complex", "business center", "skyscraper",
    "retail pavilion", "shopping center", "mixed-use development",

    # Cultural/Public (Expanded)
    "museum", "art gallery", "concert hall", "theater", "library", "cultural center",
    "government building", "civic center", "courthouse", "parliament building",
    "public archive", "philharmonic hall", "opera house", "national monument",
    "exhibition center", "community center", "performing arts complex", "city hall",
    "embassy", "consulate", "public observatory", "memorial building", "historical institute",
    "public auditorium", "urban forum", "national library", "public records office",
    "legislative assembly", "senate building", "public broadcast center", "ceremonial hall",

    # Educational
    "university building", "research facility", "academic center", "science institute",
    "campus hub", "educational laboratory", "lecture hall complex", "learning commons",
    "scholastic pavilion", "educational incubator", "student union", "academic library",

    # Residential
    "apartment building", "residential tower", "condominium complex", "housing development",

    # Special Purpose
    "innovation hub", "technology center", "convention center", "transportation terminal",
    "public transit hub", "intermodal station", "airport terminal", "maritime terminal",
    "healthcare facility", "public health center", "wellness complex", "rehabilitation center",
    "sports arena", "aquatic center", "olympic facility", "recreation complex"
]

architectural_styles = [
    # None:
    "","",

    # Contemporary Styles
    "modern", "contemporary", "minimalist", "high-tech", "sustainable", "neo-futurist",
    "parametric", "deconstructivist", "postmodern",

    # Public Building Specific
    "civic monumentalism", "democratic modernism", "institutional brutalism", "public formalism",
    "contemporary classicism", "new civic vernacular", "transparent governance", "symbolic expressionism",
    "contextual regionalism", "narrative architecture", "cultural symbolism", "heritage-inspired",

    # Regional/Historical Influences
    "Nordic minimalist", "Japanese-inspired", "Mediterranean modern", "industrial",
    "neo-vernacular", "critical regionalism", "contextual modernism", "heritage referential",
    "adaptive historical", "locally-responsive", "climate-adaptive", "cultural fusion",

    # Material-focused
    "glass and steel", "concrete", "timber-framed", "stone-clad", "prefabricated",
    "monolithic stone", "civic masonry", "ceremonial concrete", "institutional materiality",

    # Design Approach
    "biophilic", "brutalist", "organic", "geometric", "sculptural", "modular",
    "participatory design", "community-integrated", "socially responsive", "culturally expressive",
    "spatially democratic", "accessibility-focused", "inclusive design", "multi-sensory"
]

materials = [
    # None:
    "","","",

    # Standard Contemporary
    "glass curtain walls", "reflective glass panels", "concrete", "exposed concrete",
    "weathered steel", "cor-ten steel", "titanium panels", "aluminum cladding",
    "terra cotta panels", "brick masonry", "stone cladding", "timber elements",
    "composite panels", "perforated metal screens", "marble facing", "copper cladding",
    "zinc panels", "brushed steel", "polished granite", "transparent glass", "bamboo screens",

    # Civic/Public Materials
    "monumental stone", "civic granite", "ceremonial marble", "institutional limestone",
    "commemorative bronze", "symbolic brass", "democratic travertine", "cultural terrazzo",
    "heritage brick patterns", "local quarried stone", "regional material expressions",
    "traditional craftsmanship in modern context", "artisanal tile work", "community-sourced materials",
    "locally-significant natural elements", "historically-referenced masonry patterns",
    "public art integration materials", "interactive material surfaces", "acoustic material solutions"
]

features = [
    # None:
    "","","",
    # Facade Elements
    "rhythmic window patterns", "horizontal louvers", "vertical fins", "double-skin facade",
    "perforated screens", "cellular pattern facade", "pixelated facade", "staggered balconies",
    "civic-scale colonnade", "monumental entrance", "symbolic relief work", "institutional portico",
    "ceremonial steps", "plaza-integrated facade", "public-facing transparency", "democratic frontage",

    # Public Space Integration
    "public plaza integration", "urban room", "civic forecourt", "community gathering spaces",
    "transitional public zones", "semi-public courtyards", "permeable boundaries", "urban parlors",
    "civic pocket parks", "community exhibition spaces", "public observation platforms",

    # Interior Public Features
    "grand civic hall", "public atrium", "multi-height lobbies", "ceremonial staircase",
    "community forum space", "transparent governance chambers", "acoustic performance spaces",
    "democratic assembly rooms", "public viewing galleries", "interactive exhibition areas",

    # Roof/Top Elements
    "cantilevered roof", "sculptural roof form", "rooftop garden", "green roof", "solar panels",
    "skylight system", "observation deck", "terraced setbacks", "roof pavilion",
    "iconic dome", "symbolic spire", "public roof terrace", "ceremonial top feature",

    # Structure/Form Elements
    "dramatic cantilevers", "atrium spaces", "geometric volumetric play", "floating volumes",
    "suspended elements", "transparent corners", "integrated greenery", "layered forms",
    "symbolic structural forms", "democratic spatial organization", "processional sequences",
    "hierarchical spatial arrangements", "axial organization", "radial planning", "nodal spaces"
]

surroundings = [
    # None:
    "",

    "neutral background", "minimalist setting", "clean backdrop", "plain environment",
    "simple plaza foreground", "uncluttered surroundings", "monochromatic background",
    "abstract setting", "gradient background", "architectural base platform",
    "subtle landscape elements", "minimal context", "clean podium base"
]

views = [
    "front elevation", "side elevation",
    "frontal view", "direct view", "primary elevation", "full facade view"
]

lighting_conditions = [
    "even daylight", "bright and shadowless illumination", "uniform lighting",
    "soft diffused lighting", "clear daylight", "bright consistent illumination",
    "balanced lighting", "studio lighting", "neutral illumination"
]

floor_counts = [
    "1-2", "2-3", "4-6", "8-10", "12-15", "16-20", "25-30", "30-40", "multiple",
    "varying", "cascading", "terraced"
]

# The negative prompt (constant for all prompts)
negative_prompt = (
    "cropped, cut-off edges, incomplete, partial view, close-up, zoomed-in, interior view, dark, blurry, obstructed, "
    "multiple buildings, two buildings, building complex, building cluster, "
    "heavy shadows, deep shadows, dramatic shadows, chiaroscuro lighting, "
    "extreme perspective, fish-eye lens, wide-angle distortion, oblique angle, bird's eye view, worm's eye view, "
    "cluttered background, busy foreground, excessive detail, background clutter, "
    "low contrast, low resolution, poor lighting, underexposed, overexposed, "
    "artistic rendering, painterly style, sketch, illustration, "
    "text, watermark, signature, frame, border"
)

# Function to optimize prompts with architectural terms
def get_optimized_prompt(base_prompt):
    """Enhance prompt with architectural photography terms while respecting token limits"""
    # Shorter quality suffixes to avoid token limit
    quality_suffixes = [
        "architectural photography",
        "perfect composition, photorealistic",
        "clear lighting, high quality",
        "sharp focus, professional", "high resolution"
    ]

    suffix = random.choice(quality_suffixes)
    # Add explicit "single building" to the prompt
    enhanced_prompt = f"{base_prompt}"

    # Estimate token count (rough approximation: ~1.3 tokens per word)
    word_count = len(enhanced_prompt.split())
    estimated_tokens = int(word_count * 1.3)

    # If prompt is likely too long, simplify it
    if estimated_tokens > 70:  # Setting safety margin below 77
        # First, try with a shorter suffix
        enhanced_prompt = f"{base_prompt}, viewed from a distance to clearly capture the entire structure from foundation to roof."
        word_count = len(enhanced_prompt.split())
        estimated_tokens = int(word_count * 1.3)

        # If still too long, truncate the base prompt
        if estimated_tokens > 70:
            words = base_prompt.split()
            # Keep removing words from the middle until we're under the limit
            while int(len(words) * 1.3) > 55:  # Target lower to leave room for "single building" text
                # Remove from middle (keep building description and quality parts)
                middle_index = len(words) // 2
                words.pop(middle_index)
            simplified_base = " ".join(words)
            enhanced_prompt = f"{simplified_base}, viewed from a distance to clearly capture the entire structure from foundation to roof."

    return enhanced_prompt

# Generate improved prompts
prompts = []
for i in range(200):

    building_type = random.choice(building_types)
    style = random.choice(architectural_styles)
    material = random.choice(materials)
    feature = random.choice(features)
    surrounding = random.choice(surroundings)
    view = random.choice(views)
    lighting = random.choice(lighting_conditions)
    floor_count = random.choice(floor_counts)

    base_prompt = (
        f"A complete architectural exterior photograph of a single {style} {building_type} with {floor_count} floors, viewed from a distance to clearly capture the entire structure from foundation to roof. "
        f"The building is constructed with {material} and features {feature}. "
        f"Positioned in {surrounding} and ensure its full height is visible. {view} perspective, centered in frame with proper scale and proportions. "
        f"{lighting} evenly without harsh shadows. "
        f"Professional architectural photography with realistic details, proper scale, no cropping of the building edges, and natural perspective."
    )

    # Optimize the prompt
    prompt = get_optimized_prompt(base_prompt)
    prompts.append(prompt)

for i in prompts:
  print(i)

A complete architectural exterior photograph of a single institutional materiality condominium complex with 30-40 floors, viewed from a distance to clearly capture evenly without harsh shadows. Professional architectural photography with realistic details, proper scale, no cropping of the building edges, and natural perspective., viewed from a distance to clearly capture the entire structure from foundation to roof.
A complete architectural exterior photograph of a single community-integrated city hall with 8-10 floors, viewed from a distance to clearly capture the evenly without harsh shadows. Professional architectural photography with realistic details, proper scale, no cropping of the building edges, and natural perspective., viewed from a distance to clearly capture the entire structure from foundation to roof.
A complete architectural exterior photograph of a single contemporary classicism academic center with terraced floors, viewed from a distance to clearly capture evenly with

In [8]:
# Sample generation with optimal settings
def generate_test_image(prompt, negative_prompt):
    """Generate a test image with optimal settings"""
    return pipe(
        prompt=get_optimized_prompt(prompt),
        negative_prompt=negative_prompt,
        width=1024,      # Higher resolution for better details
        height=576,      # 16:9 aspect ratio
        num_inference_steps=40,  # More steps for better quality
        guidance_scale=7.5,
        generator=torch.Generator().manual_seed(42)  # Fixed seed for reproducibility
    ).images[0]

In [9]:
# Image generation settings
image_width = 1024
image_height = 576  # 16:9 aspect ratio
num_steps = 40
guidance_scale = 7

# Generate and plot images
# Adjust the subplots to accommodate all 50 images (e.g., 5 rows, 10 columns)
fig, axs = plt.subplots(10, 10, figsize=(25, 50))  # Changed to 5 rows, 10 columns
axs = axs.flatten()
# prompts = prompts[:2]

print(f"Generating {len(prompts)} building images...")

for i, prompt in enumerate(prompts):
    print(f"Generating image {i+1801}/{len(prompts)}")
    print(f"Prompt: {prompt[:100]}...")

    # Set a specific seed for reproducibility but allow variation
    seed = random.randint(1, 10000)

    # Generate the image
    image = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        width=image_width,
        height=image_height,
        num_inference_steps=num_steps,
        guidance_scale=guidance_scale,
        generator=torch.Generator().manual_seed(seed)
    ).images[0]

    # Save image
    save_path = os.path.join(save_dir, f"dt_{i+500}.png")
    image.save(save_path)

    # Display image
    axs[i].imshow(np.array(image))  # Now 'i' will be within the bounds of 'axs'
    axs[i].axis('off')
    axs[i].set_title(f"Image {i+1801}", fontsize=10)

# plt.tight_layout()
# plt.savefig(os.path.join(save_dir, "building_grid.png"), dpi=300, bbox_inches='tight')
# plt.show()

print(f"Image generation complete. All {len(prompts)} images saved to: {save_dir}")
print(f"A grid of all images has been saved as 'building_grid.png'")

# Display sample prompts
print("\n=== SAMPLE PROMPTS ===")
for i in range(min(3, len(prompts))):
    print(f"{i+500}. {prompts[i]}")
    print("-" * 80)

Output hidden; open in https://colab.research.google.com to view.